In [ ]:
from transformers import pipeline
import json

# Initialize the Korean model for text generation
model = pipeline('text-generation', model='Bllossom/llama-3.2-Korean-Bllossom-3B', device=0)

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
    - Hot Drinks: 허브티 (always served hot)
    - Iced Only Drinks: 토마토주스, 키위주스, 망고스무디, 딸기스무디, 레몬에이드, 복숭아아이스티 (always served iced)
    - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
    - Specialty Drinks: 초콜릿라떼 (available in both hot and iced versions)

    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".
    - **Current Conversation History** is a single-line cumulative log of all customer requests so far in this session. starting from 1
    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔 (핫, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```

    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -아메리카노 4잔(핫, 미디움)
      -카페라떼 라지 2잔 (핫, 라지)"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 주문하신 내용은 다음과 같습니다:
      -라떼 1잔 (아이스, 미디움)"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.

    Based on this information,
    (keep in mind that) Each JSON output should only contain the items directly requested in **Customer's New Input**, not a full history.
    if **Current Conversation History**:
    {formatted_history}

    based on **Cumulative Order Summary So Far**:
    {formatted_summary}

    and **Customer's New Input**: "{user_input}"

    generate the appropriate natural language response and JSON output
    """
    return base_prompt.strip()

# Function to get response from the Korean model pipeline
def get_response(prompt):
    response = model(prompt, max_new_tokens=150, num_return_sequences=1, temperature=0.1, top_p=0.9, truncation=True)
    return response[0]['generated_text']

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue


    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Add the latest user input to the conversation history
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    input_counter += 1

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Save natural language order summary to `summary_history`
    if natural_language_response:
        summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
        summary_history.append(summary_line)

    # Print the final natural language response for the customer
    #print("Kiosk:", natural_language_response)
    # Print only the Natural Language Response and JSON Output in the required format
    print("**Natural Language Response**:")
    print(natural_language_response)
    print("\n**JSON Output**:")
    print(json.dumps(json_output, indent=2, ensure_ascii=False))  # Pretty-print JSON output

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history and summary history for a reset
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")


In [1]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!git config --global user.name "fatih0122"
!git config --global user.email "m.fatih012001@gmail.com"

In [4]:
!git clone https://github.com/AI-coffee-Kiosk/Prompting-for-llama.git


Cloning into 'Prompting-for-llama'...


In [8]:

!cp "/content/promptforkrmodel.ipynb" "/content/Prompting-for-llama/"


cp: cannot stat '/content/promptforkrmodel.ipynb': No such file or directory
